In [1]:
from astropy.table import Table
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import fitsio
import desimodel.io
import desitarget.mtl
import desisim.quickcat
from astropy.io import fits
from astropy.table import Table, Column, vstack
import json
import shutil
import healpy

from desitarget.targetmask import desi_mask, obsconditions
from collections import Counter
import subprocess

%matplotlib inline

In [20]:
initial_mtl_file = "targets/subset_dr8_mtl_dark_gray_NGC.fits"
pixweight_file = "/project/projectdirs/desi/target/catalogs/dr8/0.31.1/pixweight/pixweight-dr8-0.31.1.fits"
initial_truth_file = "targets/subset_truth_dr8_mtl_dark_gray_NGC.fits"

In [21]:
truth = Table.read(initial_truth_file)
initial_mtl = Table.read(initial_mtl_file)

In [22]:
truth[:10]

TARGETID,MOCKID,TRUEZ,TRUESPECTYPE,TEMPLATETYPE,TEMPLATESUBTYPE,TEMPLATEID,SEED,MAG,MAGFILTER,FLUX_G,FLUX_R,FLUX_Z,FLUX_W1,FLUX_W2,FLUX_W3,FLUX_W4
,,,,,,,,mag,,nanomaggies,nanomaggies,nanomaggies,nanomaggies,nanomaggies,nanomaggies,nanomaggies
int64,int64,float32,str10,str10,str10,int32,int64,float32,str15,float32,float32,float32,float32,float32,float32,float32
35185748411940893,35185748411940893,0.2,GALAXY,BGS,,-1,-1,0.0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35185748411940915,35185748411940915,0.0,STAR,STAR,,-1,-1,0.0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35185748411940916,35185748411940916,0.0,STAR,STAR,,-1,-1,0.0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35185748411940925,35185748411940925,0.0,STAR,STAR,,-1,-1,0.0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35185748411940927,35185748411940927,1.5,GALAXY,ELG,,-1,-1,0.0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35185748411940947,35185748411940947,1.5,GALAXY,ELG,,-1,-1,0.0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35185748411940954,35185748411940954,1.5,GALAXY,ELG,,-1,-1,0.0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35185748411940960,35185748411940960,1.5,GALAXY,ELG,,-1,-1,0.0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
initial_mtl[:10]

RA,DEC,TARGETID,DESI_TARGET,BGS_TARGET,MWS_TARGET,PRIORITY_INIT,SUBPRIORITY,NUMOBS_INIT,HPXPIXEL,NUMOBS_MORE,PRIORITY,OBSCONDITIONS
float64,float64,int64,int64,int64,int64,int64,float64,int64,int64,int64,int64,int32
163.252082455,-0.624211104073,35185748411940893,1152921504606846976,65537,0,2000,0.751984172406,1,27023,0,0,7
163.413737215,-0.622343341717,35185748411940915,2305843009213693952,0,10240,1500,0.679526087745,1,27023,1,1500,7
163.414541574,-0.622295995581,35185748411940916,2305843009213693952,0,10240,1500,0.374817713069,1,27023,1,1500,7
163.277324683,-0.621896373342,35185748411940925,2305843009213693952,0,1280,1500,0.491139885087,1,27023,1,1500,7
163.473522103,-0.622380001889,35185748411940927,131074,0,0,3000,0.853900531068,1,27023,1,3000,3
163.495674904,-0.621547358804,35185748411940947,131074,0,0,3000,0.978607452713,1,27023,1,3000,3
163.324883091,-0.620610593885,35185748411940954,131074,0,0,3000,0.87926552397,1,27023,1,3000,3
163.407663822,-0.620698404077,35185748411940960,131074,0,0,3000,0.653314844572,1,27023,1,3000,3
163.406836857,-0.621139978782,35185748411940961,131074,0,0,3000,0.694506552792,1,27023,1,3000,3


In [24]:
ii_truth_qso = truth['TRUESPECTYPE']=='QSO'
high_z_qso = truth['TRUEZ'] > 2.15
print(np.count_nonzero(ii_truth_qso & high_z_qso), np.count_nonzero(ii_truth_qso & ~high_z_qso))

45692 201221


In [25]:
area_approx = (initial_mtl['RA'].max() - initial_mtl['RA'].min()) * (initial_mtl['DEC'].max() - initial_mtl['DEC'].min())

In [26]:
print(area_approx * 50)

44999.958658
